In [ ]:
!pip install gymnasium

In [ ]:
import gymnasium as gym
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import deque
import random
from tqdm import tqdm

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# Define model
class QNetwork(nn.Module):

    def __init__(self, state_shape, h1_nodes, action_space_size, seed):
        """ state_shape (int): Dimension of each state
            h1_nodes (int): Number of nodes in first hidden layer
            action_space_size (int): Dimension of each action
            seed (int): Random seed
        """
        super(QNetwork, self).__init__()
        self.seed = torch.manual_seed(seed)
         ## add three fully connected layers using nn.Linear()
         #-----------------add code here-----------------------


    def forward(self, state):
      # define the forward propagation  by adding the relu() activation function between layers
      #-----------------add code here----------------------- 
      return x

In [ ]:
# Define memory for Experience Replay
class ReplayMemory():
    def __init__(self, maxlen, batch_size):
        self.memory = deque([], maxlen=maxlen)
        self.batch_size = batch_size

    def append(self, experience):
        self.memory.append(experience)

    def sample(self):
        #Randomly sample a batch of experiences from memory 
        #-----------------add code here-----------------------

    def __len__(self):
        return len(self.memory)

In [ ]:
#Define the Deep QLearning Agent
class DQAgent():
        """Interacts with and learns from the environment."""

        def __init__(self, state_shape, action_space_size, seed, h1_nodes=64, learning_rate=5e-4, replay_memory_size=int(1e5),batch_size=64, UPDATE_EVERY=20, LEARN_EVERY=4,GAMMA=0.99):
           """Initialize an Agent object. """

           self.state_shape = state_shape
           self.action_space_size = action_space_size
           self.seed = random.seed(seed)
           self.GAMMA = GAMMA
           self.UPDATE_EVERY = UPDATE_EVERY
           self.LERN_EVERY = LEARN_EVERY
           self.h1_nodes = h1_nodes
           self.learning_rate = learning_rate
           self.batch_size = batch_size
           self.replay_memory_size = replay_memory_size


           # Q-Network
           # create the local and the target networks
           self.local_dqn =  #--add code here---
           self.target_dqn =  #--add code here---

           self.optimizer = optim.Adam(self.local_dqn.parameters(), lr=self.learning_rate)

           # Replay memory
           self.memory = ReplayMemory( self.replay_memory_size,self.batch_size)

           # Initialize time step (for learn every LEARN_EVERY steps)
           self.l_step = 0

           # Initialize time step (for updating every UPDATE_EVERY steps)
           self.t_step = 0
            
        def state_to_dqn_input(self, state)->torch.Tensor:
           return torch.from_numpy(state).float().unsqueeze(0).to(device)

        #Returns actions for given state
        #Epsilon-greedy action selection
        def act(self, state, eps=0.):
          if random.random() > eps:
            #--add code here---
          else:
            #--add code here---

        def step(self, state, action, reward, next_state, done):
           # Save experience in replay memory
           #----------add code here---

           # If enough samples are available in memory, get random subset and learn
           self.l_step = (self.l_step + 1) % self.LERN_EVERY
           if self.l_step == 0 and len(self.memory) > self.batch_size:
                experiences = #--add code here---
                #--add code here---

           # Copy local network to target network every UPDATE_EVERY time steps
           self.t_step = (self.t_step + 1) % self.UPDATE_EVERY
           if self.t_step == 0:
              #--add code here---

        def learn(self, experiences):
          q_target_list = []
          q_expected_list = []
          # Obtain random minibatch of tuples from ReplayMemory
          for  state, action, reward, next_state, done in experiences:
          
             # Convert state, next_state, action abd reward to tensors 
             state =#--add code here---
             next_state = #--add code here---
             action= torch.from_numpy(np.array([action])).long().unsqueeze(0).to(device)
             reward = torch.from_numpy(np.array([reward])).float().to(device)
             ## Compute and minimize the loss

             ### Extract next maximum estimated value from target network
             if(done):
               q_target =  #--add code here---
             else:
               q_target=  #--add code here---

             ### Calculate expected value from local network
             q_local = #--add code here---

             q_target_list.append(q_target)
             q_expected_list.append(q_local)

          ### Loss calculation (we used Mean squared error)
          loss = F.mse_loss(torch.cat(q_expected_list), torch.cat(q_target_list))
          self.optimizer.zero_grad()
          loss.backward()
          self.optimizer.step()


        

In [ ]:
def dqn_train(agent, n_episodes=300, max_t=1000, eps_start=1.0, eps_end=0.01, eps_decay=0.995):

    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    eps = eps_start                    # initialize epsilon

    for i_episode in  tqdm(range(1, n_episodes+1)):
        state = env.reset()[0]

        score = 0
        for t in range(max_t):
            # select an action
            action = #--add code here---

            #apply the selected action
            next_state, reward, done, _,_=env.step(action)
            
            # store the current experience
            #--add code here---

            state = next_state
            score += reward
            if done:
                break
                
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        eps = max(eps_end, eps_decay*eps) # decrease epsilon

        if i_episode % 50 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))

    torch.save(agent.local_dqn.state_dict(), 'checkpoint.pth')
    return scores

In [ ]:
env = gym.make('CartPole-v1')
print(env.reset())

print('State shape: ', env.observation_space.shape[0])
print('Number of actions: ', env.action_space.n)

In [ ]:
agent = DQAgent(state_shape=env.observation_space.shape[0], action_space_size=env.action_space.n, seed=0)
scores = dqn_train(agent)

In [ ]:
scores = pd.Series(scores, name="scores")
scores.describe()
fig, ax = plt.subplots(1, 1)
_ = scores.plot(ax=ax, label="Scores")
_ = (scores.rolling(window=100)
           .mean()
           .rename("Rolling Average")
           .plot(ax=ax))
ax.legend()
_ = ax.set_xlabel("Episode Number")
_ = ax.set_ylabel("Score")